In [36]:
# Here's an example of basic streaming applied to different use cases:

import openai
import instructor
import time
from pprint import pprint
from pydantic import BaseModel
from IPython.display import clear_output

client = instructor.from_openai(openai.OpenAI())

class ChatResponse(BaseModel):
    content: str
    follow_up_question: list[str]

resp = client.chat.completions.create_partial(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant"},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    response_model=ChatResponse,
    stream=True
)

for chunk in resp:
    clear_output(wait=True)
    pprint(chunk.model_dump())
    time.sleep(0.05)

{'content': 'The capital of France is Paris.',
 'follow_up_question': ['What is the population of Paris?',
                        'What are some famous landmarks in Paris?',
                        'What cuisine is popular in France?']}


In [37]:
# Citations


class Citation(BaseModel):
    id: str
    title: str

class ChatResponse(BaseModel):
    content: str
    citations: list[Citation]

resp = client.chat.completions.create_partial(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant. When providing information, cite your sources using markdown-like notation, e.g., [string](1), [string](2), etc."},
        {"role": "user", "content": """

        Here are some relevant sources:
         
        <chunks>
        <chunk>
        (1) Eiffel Tower: An iconic iron lattice tower that has become a global cultural icon of France.
        </chunk>
        <chunk>
        (2) Louvre Museum: Home to many famous works of art, including the Mona Lisa.
        </chunk>
        <chunk>
        (3) Notre-Dame Cathedral: A medieval Catholic cathedral known for its French Gothic architecture and its prominent role in Victor Hugo's novel "The Hunchback of Notre-Dame".
        </chunk>

        </chunks>
         
        <citation_examples>
            To cite a source, use markdown-style link notation. For example: [relevant text](source number). Citations should be integrated naturally within the sentence, not just appended at the end. This approach ensures a smooth flow of information while providing proper attribution. For instance: "The [1980 'Miracle on Ice'](1) is considered one of the greatest upsets in Olympic history" is preferable to "There was a great upset in Olympic history. [Miracle on Ice](1)".
        </citation_examples>
         
         Answer the question using the sources provided. using markdown notation for citation
         """},
        {"role": "user", "content": "What are the main attractions in Paris?"},
    ],
    response_model=ChatResponse,
    stream=True,
)

for chunk in resp:
    clear_output(wait=True)
    pprint(chunk.model_dump())
    time.sleep(0.1)

{'citations': [{'id': '1', 'title': 'Eiffel Tower'},
               {'id': '2', 'title': 'Louvre Museum'},
               {'id': '3', 'title': 'Notre-Dame Cathedral'}],
 'content': "Paris, often referred to as the 'City of Light,' is home to "
            'several world-famous attractions that draw millions of visitors '
            'each year. One of the most iconic landmarks is the [Eiffel '
            'Tower](1), an iron lattice tower that has become a global symbol '
            'of France. Art enthusiasts are naturally drawn to the [Louvre '
            'Museum](2), which houses many renowned works of art, including '
            "Leonardo da Vinci's Mona Lisa. Another significant attraction is "
            'the [Notre-Dame Cathedral](3), a stunning example of French '
            "Gothic architecture that also gained fame through Victor Hugo's "
            'novel "The Hunchback of Notre-Dame."'}


In [39]:
# Tools

from typing import Optional, Literal
from pydantic import BaseModel

class SearchQuery(BaseModel):
    query: str
    start_date: Optional[str]
    end_date: Optional[str]

resp = client.chat.completions.create_partial(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an AI assistant that understands and interprets search queries. Your task is to parse the user's input into a structured SearchQuery and provide an explanation of your interpretation. the date is 2024-05-15"},
        {"role": "user", "content": "Find documents about project XYZ's retainer program from last week"},
    ],
    response_model=SearchQuery,
    stream=True,
)

for chunk in resp:
    clear_output(wait=True)
    print(chunk.model_dump_json(indent=2))
    time.sleep(0.1)

{
  "query": "project XYZ's retainer program",
  "start_date": "2024-05-08",
  "end_date": "2024-05-14"
}
